# Network from OSM download
This tutorial shows how to download and configure a road network from OpenStreetMap (OSM) and process it using the RA2CE library.

## Step 1. Import the Required Packages

In [1]:
from pathlib import Path
import geopandas as gpd
from shapely.geometry.polygon import Polygon

from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.network_config_data import NetworkSection, NetworkConfigData
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.ra2ce_handler import Ra2ceHandler

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2. Define a Region of Interest
Create a bounding box for the area of interest and save it as a geojson file.

In [ ]:
root_dir = Path("data", "network_from_OSM")
network_path = root_dir / "network"

polygon = Polygon([
    [4.925796685034555, 52.15567004009617],
    [4.925796685034555, 51.969875228118696],
    [5.263478289905265, 51.969875228118696],
    [5.263478289905265, 52.15567004009617],
    [4.925796685034555, 52.15567004009617]
])

# Convert polygon into geojson file
gdf_polygon = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[polygon])
gdf_polygon_path = network_path.joinpath("polygon.geojson")
gdf_polygon.to_file(gdf_polygon_path, driver="GeoJSON")

## Step 3. Network Configuration
Define the network configuration using [NetworkConfigData](../api/ra2ce.network.network_config_data.html#ra2ce.network.network_config_data.network_config_data.NetworkConfigData){.api-ref} and [NetworkSection](../api/ra2ce.network.network_config_data.html#ra2ce.network.network_config_data.network_config_data.NetworkSection){.api-ref} classes., specifying OSM as the source and selecting road types.

In [ ]:
network_section = NetworkSection(
    source=SourceEnum.OSM_DOWNLOAD,
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[RoadTypeEnum.MOTORWAY, RoadTypeEnum.PRIMARY],
    polygon=gdf_polygon_path,
    save_gpkg=True,
)

network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir / "static",
    network=network_section,
)

## Step 4. Initialize and Configure RA2CE
Generate the network and store results in `static/output_graph`.

In [ ]:
handler = Ra2ceHandler.from_config(network=network_config_data, analysis=None)
handler.configure()

## Step 5. Load and Inspect the Output

In [ ]:
path_output_graph = root_dir.joinpath("static", "output_graph")
base_graph_edges = path_output_graph.joinpath("base_graph_edges.gpkg")
edges_gdf = gpd.read_file(base_graph_edges, driver="GPKG")

base_graph_nodes = path_output_graph.joinpath("base_graph_nodes.gpkg")
nodes_gdf = gpd.read_file(base_graph_nodes, driver="GPKG")

## Step 6. Plot Nodes and Edges

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 15))

# Plot edges first
baseplot = edges_gdf.plot(ax=ax, color="grey")

# Overlay nodes
nodes_gdf.plot(ax=baseplot, color="blue", markersize=20)

plt.show()

![RA2CE OSM-based network](/_resources/figures/network_osm.png)